<img src = "https://escp.eu/sites/default/files/logo/ESCP-logo-white-misalign.svg" width = 400 style="background-color: #240085;">
<h1 align=center><font size = 6>ESCP Business School</font></h1>
<h3 align=center><font size = 5>ESCP Hackathon</font><br/>
<font size = 3>Project 3: Brand sentiment analysis | Criteo</font></h3>
<h6 align=center>Chapter 1a: Sentiment analysis</h6>

Last Updated: March 16, 2021\
Author: Group 07 - Arya Bhattacharya, Hanadi Yachoui, Ming Hin Chung, Resha Dirga

<h3>Chapter Objectives</h3>

<p>This chapter aims to perform sentiment analysis on each of the tweets that has been retrieved.</p>

In [1]:
# # Install dependencies
# !pip install numpy
# !pip install pandas
# !pip install time
# !pip install requests
# !pip install monkeylearn

In [2]:
# Import necessary modules
import numpy as np
import pandas as pd
import time

import requests
from monkeylearn import MonkeyLearn

In [3]:
# Read tweet data
df = pd.read_csv('tweets_202203040000.csv')
df

,Unnamed: 0,id,created_at,in_reply_to_user_id,author_id,text,mentions,source,lang,text_clean
0,0,1503882954375929866,Tue Mar 15 23:56:51 +0000 2022,NaN,612834201,RT @WrocWW: @AnonOpsSE I found few words to #L...,"['WrocWW', 'AnonOpsSE']",Twitter Web App,en,@WrocWW: @AnonOpsSE I found few words to #Lero...
1,0,1503882686380879877,Tue Mar 15 23:55:47 +0000 2022,NaN,612834201,"RT @Ro_manoff: @tashecon Also, please pay atte...","['Ro_manoff', 'tashecon']",Twitter Web App,en,"@Ro_manoff: @tashecon Also, please pay attenti..."
2,0,1503881899328032771,Tue Mar 15 23:52:40 +0000 2022,NaN,4174651769,RT @NoButYeahBut_: #BoycottDecathlon!!!\n#Deca...,['NoButYeahBut_'],Twitter for Android,en,@NoButYeahBut_: #BoycottDecathlon!!!\n#Decathl...
3,0,1503880374874607620,Tue Mar 15 23:46:36 +0000 2022,126267113.0,51585491,@Decathlon @DecathlonUK why are you still doin...,"['Decathlon', 'DecathlonUK']",Twitter for iPhone,en,@Decathlon @DecathlonUK why are you still doin...
4,0,1503880275318775809,Tue Mar 15 23:46:13 +0000 2022,NaN,4174651769,"RT @TrudeauMaggie: Auchan, Leroy Merlin, Decat...",['TrudeauMaggie'],Twitter for Android,en,"@TrudeauMaggie: Auchan, Leroy Merlin, Decathlo..."
...,...,...,...,...,...,...,...,...,...,...
3569,0,1499543192596725760,Fri Mar 04 00:32:11 +0000 2022,NaN,360737613,RT @BoosterVista: Kudos to the MVHS Academic D...,['BoosterVista'],Twitter for iPhone,en,@BoosterVista: Kudos to the MVHS Academic Deca...
3570,0,1499541366673747971,Fri Mar 04 00:24:56 +0000 2022,NaN,3316370737,"RT @jwilliams_mhs: As a team, MHS Academic Dec...",['jwilliams_mhs'],Twitter for iPhone,en,"@jwilliams_mhs: As a team, MHS Academic Decath..."
3571,0,1499541283886485507,Fri Mar 04 00:24:36 +0000 2022,NaN,768215814520524801,"As a team, MHS Academic Decathlon came away fr...",[],Twitter for iPhone,en,"As a team, MHS Academic Decathlon came away fr..."
3572,0,1499538730570465284,Fri Mar 04 00:14:28 +0000 2022,NaN,1488628891916849156,33239. \nTriathlon n. Athletic contest of thr...,[],botjeelaa,en,33239. \nTriathlon n. Athletic contest of thr...


In [4]:
# Setup parameters for sentiment analysis
list_of_texts = df['text_clean']
diff_threshold = 0
neutral_margin = 0.10

In [5]:
# Create a function to send query to Hugging Face
def query(payload):
    response = requests.post(HF_API_URL, headers=hf_headers, json=payload)
    return response.json()

In [6]:
# Authentications
HF_API_URL = "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english"
HF_API_TOKEN = # Insert your Hugging Face API Token here
hf_headers = {"Authorization": f"Bearer {HF_API_TOKEN}"}

ML_API_TOKEN = # Insert your MonkeyLearn API Token here
ml_headers = {"Authorization": f"Bearer {ML_API_TOKEN}"}

In [7]:
# Create a function to perform sentiment analysis
def analyse_sentiment(text):
    positives = []
    negatives = []
    # Hugging face
    output = query({
      "inputs": text,
    })

    # Monkey learn result
    try:
        ml = MonkeyLearn(ML_API_TOKEN)
        data = [text]
        model_id = 'cl_pi3C7JiL'
        result = ml.classifiers.classify(model_id, data)

        ml_sentiment = result.body[0]['classifications'][0]['tag_name'].lower()
        ml_value = result.body[0]['classifications'][0]['confidence']

        if ml_sentiment == 'positive':
            positives.append(ml_value)
            negatives.append(1 - ml_value)
        elif ml_sentiment == 'negative':
            positives.append(1 - ml_value)
            negatives.append(ml_value)
    except:
        pass

    # Hugging face (cont'd)
    while len(output) < 1:
        time.sleep(1)

    response = output[0]
    n_outputs = len(response)
    for n in range(0,n_outputs):
        if response[n]['label'] == 'POSITIVE':
            hf_positive = response[n]['score']
            positives.append(hf_positive)
        elif response[n]['label'] == 'NEGATIVE':
            hf_negative = response[n]['score']
            negatives.append(hf_negative)
    
    try:
        if ml_sentiment == 'neutral':
            result_positive = 0.5
            result_negative = 0.5
        else:
            result_positive = sum(positives)/len(positives)
            result_negative = sum(negatives)/len(negatives)
    except:
        result_positive = sum(positives)/len(positives)
        result_negative = sum(negatives)/len(negatives)

    diff = result_positive - result_negative

    if diff > diff_threshold + neutral_margin:
        result_sentiment = 'positive'
    elif diff < diff_threshold - neutral_margin:
        result_sentiment = 'negative'
    else:
        result_sentiment = 'neutral'

    analysis_result = {
      'sentiment': result_sentiment,
      'positive' : result_positive,
      'negative' : result_negative
    }

    return analysis_result

In [8]:
# Prepare variables for sentiment analysis
sentiment_list = []
positive_conf = []
negative_conf = []

In [9]:
# Perform sentiment analysis
for text in list_of_texts:
    # print(str(list_of_texts.index(text)+1) + '/' + str(len(list_of_texts)))
    sentiment_result = analyse_sentiment(text)
    sentiment_list.append(sentiment_result['sentiment'])
    positive_conf.append(sentiment_result['positive'])
    negative_conf.append(sentiment_result['negative'])


KeyboardInterrupt: 

In [ ]:
# Review sentiment analysis result
df['Sentiment'] = sentiment_list
df['Positive_sentiment'] = positive_conf
df['Negative_sentiment'] = negative_conf
pd.set_option('display.max_colwidth',10000)
df[['text', 'Sentiment', 'Positive_sentiment', 'Negative_sentiment']]

In [ ]:
# Export sentiment analysis result
df.to_csv('tweets_analysed.csv')